In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
import urllib
from datetime import datetime
import re

server = 'AG-SERVER-043'
database = '2526 GREEK' 
username = 'data05'
password = 'sai@123'

params = urllib.parse.quote_plus(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};PWD={password}"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

csv_folder = r"E:\DATA\2025-2026\MERGE_TRADEBOOK\MERGE_GREEK"

start_date = datetime.strptime("2025-02-01", "%Y-%m-%d")
end_date = datetime.strptime("2025-02-01", "%Y-%m-%d")

def extract_date_from_filename(filename):
    match = re.search(r'MergeGreek(\d{8})', filename)  # removed underscore
    if match:
        date_str = match.group(1)
        try:
            return datetime.strptime(date_str, "%d%m%Y")
        except Exception as e:
            print(f"Date parse error in filename {filename}: {e}")
            return None
    return None

expected_cols = [
    'SourceFile', 'ExchangeTradeID', 'Symbol', 'SecurityType', 'ExpiryDate', 'StrikePrice',
    'OptionType', 'SecurityName', 'ManagerID', 'Side', 'Quantity', 'Price', 'ClientID',
    'MemberID', 'ExchangeOrderNo', 'ExchangeOrderStatus', 'Code', 'Exchange', 'TradeDateTime'
]

# First, find all files within the date range
files_to_process = []
for file in os.listdir(csv_folder):
    if file.lower().endswith('.csv'):
        file_date = extract_date_from_filename(file)
        if file_date is not None and start_date <= file_date <= end_date:
            files_to_process.append(file)

if files_to_process:
    # Truncate table only if files exist to process
    with engine.connect() as connection:
        connection.execute(text("TRUNCATE TABLE Upload_Staging;"))
        print("Upload_Staging table truncated (cleared).")

    # Process files
    for file in files_to_process:
        file_path = os.path.join(csv_folder, file)
        print(f"\n📥 Processing file: {file_path}")
        try:
            df = pd.read_csv(file_path, quotechar='"', dtype=str)
            df['SourceFile'] = file

            # Clean column headers
            df.columns = [col.strip() for col in df.columns]

            # Check for missing columns
            missing_cols = [col for col in expected_cols if col not in df.columns]
            if missing_cols:
                print(f"Skipping {file} — missing columns: {missing_cols}")
                continue

            df = df[expected_cols]

            # Append data to SQL table
            df.to_sql(name='Upload_Staging', con=engine, if_exists='append', index=False)
            print(f"✅ Successfully appended data from {file} into Upload_Staging")

        except Exception as e:
            print(f"❌ Failed to process {file}: {e}")
else:
    print("No files found in the given date range; skipping truncation and loading.")


Upload_Staging table truncated (cleared).

📥 Processing file: E:\DATA\2025-2026\MERGE_TRADEBOOK\MERGE_GREEK\MergeGreek01022025.csv
Skipping MergeGreek01022025.csv — missing columns: ['Code', 'Exchange', 'TradeDateTime']

📥 Processing file: E:\DATA\2025-2026\MERGE_TRADEBOOK\MERGE_GREEK\MergeGreek01022025_.csv
✅ Successfully appended data from MergeGreek01022025_.csv into Upload_Staging

📥 Processing file: E:\DATA\2025-2026\MERGE_TRADEBOOK\MERGE_GREEK\MergeGreek01022025_CLEANING_LOG.csv
Skipping MergeGreek01022025_CLEANING_LOG.csv — missing columns: ['ExchangeTradeID', 'Symbol', 'SecurityType', 'ExpiryDate', 'StrikePrice', 'OptionType', 'SecurityName', 'ManagerID', 'Side', 'Quantity', 'Price', 'ClientID', 'MemberID', 'ExchangeOrderNo', 'ExchangeOrderStatus', 'Code', 'Exchange', 'TradeDateTime']
